## AutoGraph: examples of simple algorithms

This notebook shows how you can use AutoGraph to compile simple algorithms and run them in TensorFlow.

It requires the nightly build of TensorFlow, which is installed below.

In [0]:
!pip install -U -q tf-nightly-2.0-preview

In [0]:
import tensorflow as tf

tf = tf.compat.v2
tf.enable_v2_behavior()

### Fibonacci numbers

https://en.wikipedia.org/wiki/Fibonacci_number


In [11]:
@tf.function
def fib(n):
  f1 = 0
  f2 = 1
  for i in tf.range(n):
    tmp = f2
    f2 = f2 + f1
    f1 = tmp
    tf.print(i, ': ', f2)
  return f2


_ = fib(tf.constant(10))

0 :  1
1 :  2
2 :  3
3 :  5
4 :  8
5 :  13
6 :  21
7 :  34
8 :  55
9 :  89


#### Generated code

In [0]:
print(tf.autograph.to_code(fib.python_function))

### Fizz Buzz

https://en.wikipedia.org/wiki/Fizz_buzz

In [12]:
import tensorflow as tf


@tf.function(experimental_autograph_options=tf.autograph.experimental.Feature.EQUALITY_OPERATORS)
def fizzbuzz(i, n):
  while i < n:
    msg = ''
    if i % 3 == 0:
      msg += 'Fizz'
    if i % 5 == 0:
      msg += 'Buzz'
    if msg == '':
      msg = tf.as_string(i)
    tf.print(msg)
    i += 1
  return i

_ = fizzbuzz(tf.constant(10), tf.constant(16))

Buzz
11
Fizz
13
14
FizzBuzz


#### Generated code

In [0]:
print(tf.autograph.to_code(fizzbuzz.python_function))

### Conway's Game of Life

https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life

#### Testing boilerplate

In [0]:
NUM_STEPS = 1

#### Game of Life for AutoGraph

Note: the code may take a while to run.

In [0]:
#@test {"skip": true} 
NUM_STEPS = 75

Note: This code uses a non-vectorized algorithm, which is quite slow. For 75 steps, it will take a few minutes to run. 

In [10]:
import time
import traceback
import sys

from matplotlib import pyplot as plt
from matplotlib import animation as anim
import numpy as np
from IPython import display


@tf.autograph.experimental.do_not_convert
def render(boards):
  fig = plt.figure()

  ims = []
  for b in boards:
    im = plt.imshow(b, interpolation='none')
    im.axes.get_xaxis().set_visible(False)
    im.axes.get_yaxis().set_visible(False)
    ims.append([im])

  try:
    ani = anim.ArtistAnimation(
        fig, ims, interval=100, blit=True, repeat_delay=5000)
    plt.close()

    display.display(display.HTML(ani.to_html5_video()))
  except RuntimeError:
    print('Coult not render animation:')
    traceback.print_exc()
    return 1
  return 0


def gol_episode(board):
  new_board = tf.TensorArray(tf.int32, 0, dynamic_size=True)

  for i in tf.range(len(board)):
    for j in tf.range(len(board[i])):
      num_neighbors = tf.reduce_sum(
          board[tf.maximum(i-1, 0):tf.minimum(i+2, len(board)),
                tf.maximum(j-1, 0):tf.minimum(j+2, len(board[i]))]
      ) - board[i][j]
      
      if num_neighbors == 2:
        new_cell = board[i][j]
      elif num_neighbors == 3:
        new_cell = 1
      else:
        new_cell = 0
      
      new_board.append(new_cell)
  final_board = new_board.stack()
  final_board = tf.reshape(final_board, board.shape)
  return final_board
  

@tf.function(experimental_autograph_options=(
    tf.autograph.experimental.Feature.EQUALITY_OPERATORS,
    tf.autograph.experimental.Feature.BUILTIN_FUNCTIONS,
    tf.autograph.experimental.Feature.LISTS,
    ))
def gol(initial_board):
  board = initial_board
  boards = tf.TensorArray(tf.int32, size=0, dynamic_size=True)

  i = 0
  for i in tf.range(NUM_STEPS):
    board = gol_episode(board)
    boards.append(board)
  boards = boards.stack()
  tf.py_function(render, (boards,), (tf.int64,))
  return i
 

# Gosper glider gun
# Adapted from http://www.cplusplus.com/forum/lounge/75168/
_ = 0
initial_board = tf.constant((
    ( _,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_ ),
    ( _,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,1,_,_,_,_,_,_,_,_,_,_,_,_ ),
    ( _,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,1,_,1,_,_,_,_,_,_,_,_,_,_,_,_ ),
    ( _,_,_,_,_,_,_,_,_,_,_,_,_,1,1,_,_,_,_,_,_,1,1,_,_,_,_,_,_,_,_,_,_,_,_,1,1,_ ),
    ( _,_,_,_,_,_,_,_,_,_,_,_,1,_,_,_,1,_,_,_,_,1,1,_,_,_,_,_,_,_,_,_,_,_,_,1,1,_ ),
    ( _,1,1,_,_,_,_,_,_,_,_,1,_,_,_,_,_,1,_,_,_,1,1,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_ ),
    ( _,1,1,_,_,_,_,_,_,_,_,1,_,_,_,1,_,1,1,_,_,_,_,1,_,1,_,_,_,_,_,_,_,_,_,_,_,_ ),
    ( _,_,_,_,_,_,_,_,_,_,_,1,_,_,_,_,_,1,_,_,_,_,_,_,_,1,_,_,_,_,_,_,_,_,_,_,_,_ ),
    ( _,_,_,_,_,_,_,_,_,_,_,_,1,_,_,_,1,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_ ),
    ( _,_,_,_,_,_,_,_,_,_,_,_,_,1,1,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_ ),
    ( _,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_ ),
    ( _,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_ ),
))
initial_board = tf.pad(initial_board, ((0, 10), (0, 5)))

_ = gol(initial_board)

#### Generated code

In [0]:
print(tf.autograph.to_code(gol.python_function))